# GEM Radix-based

In [1]:
SCALE_CROWDING=False
PREVENT_ZERO = True

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7fe1e4d11ef0>)


### Try optimizing using radix for one condition first

In [4]:
from cobra.io import load_json_model

mdl_ref  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')
ijomc  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')

In [5]:
mu_id = 'BIOMASS_Ec_iJO1366_core_53p95M'

In [6]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.get_by_id(mu_id).x
print(mu_crowd0)

0.972916210772


In [7]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

#N_CONDS = len(conds)
N_CONDS = 1

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]
#df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'])]

## Need to allow higher growth for acetate with higher uptake rate

In [8]:
df_conds.loc[ df_conds.cond=='acetate', 'lb'] = -20

In [9]:
stacker = StackOptimizer()
stacker.stack_models(ijomc, df_conds)

In [10]:
stacker.model.optimize(solver='gurobi')

<Solution 0.97 at 0x7fe182448f10>

In [11]:
# stacker.model_dict['glucose'].optimize()
# print stacker.model_dict['glucose'].reactions.query('BIOMASS')[0].x
# print stacker.model_dict['glucose'].reactions.query('EX_glc')[0].x
# print stacker.model_dict['glucose'].reactions.query('EX_fru')[0].x

# stacker.model_dict['acetate'].optimize()
# print stacker.model_dict['acetate'].reactions.query('BIOMASS')[0].x
# print stacker.model_dict['acetate'].reactions.query('EX_glc')[0].x
# print stacker.model_dict['acetate'].reactions.query('EX_ac')[0].x
for mdl_ind,mdl in iteritems(stacker.model_dict):
    mdl.optimize()
    print('%s. mu=%g.' % (mdl_ind, mdl.solution.f))

glucose. mu=0.972916.


In [12]:
for mdl_ind,mdl in iteritems(stacker.model_dict):
    opt = Optimizer(mdl)
    gapi = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)

In [13]:
stacker.model.optimize(solver='gurobi')

<Solution 1.95 at 0x7fe180452910>

In [14]:
import numpy as np

radix = 2.
print('Radix:',radix)
powers = np.arange(-3,4)
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Discretize crowding coefficients into radix
#crowding_p = gap.metabolites.crowding
#crowding_cons = stacker.model.metabolites.query('crowding')

var_cons_dict = {}
# Get the group ID from reference model
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
for rxn_ref in crowding_ref.reactions:    
    for mdl_ind, mdl in iteritems(stacker.model_dict):
        crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
        var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
        rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
        # Get the coefficient in the dual
        cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
        a0 = rxn_p.metabolites[crowding_p]
        if var_cons_dict.has_key(rxn_ref.id):
            var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        else:
            var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

('Radix:', 2.0)
('Powers:', array([-3, -2, -1,  0,  1,  2,  3]))
('Digits:', [1.0])


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:9 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [15]:
# %lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power, prevent_zero=True)

In [16]:
%time opt.to_radix(stacker.model, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)

CPU times: user 57.8 s, sys: 1.21 s, total: 59 s
Wall time: 1min


[1.0]

In [17]:
### Try scaling
if SCALE_CROWDING:
    for mdl_ind,mdl in iteritems(stacker.model_dict):
        crowding = stacker.model.metabolites.get_by_id('crowding'+'_%s'%mdl_ind)
        bound0  = crowding._bound
        f_scale = crowding._bound
        crowding._bound = bound0 / f_scale
        for rxn in crowding.reactions:
            rxn._metabolites[crowding] = rxn._metabolites[crowding]/f_scale

In [18]:
for rxn in stacker.model.reactions:
    rxn.objective_coefficient = 0.

for group_id in var_cons_dict.keys():
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):
            yid = 'binary_%s%s%s'%(group_id,k,l)
            y   = stacker.model.reactions.get_by_id(yid)
            ### PREFER pwr=0, digit=1
            if pwr==0 and digit==1:
                #print('Preferring a0')
                y.objective_coefficient = 0.
            else:
                y.objective_coefficient = 1.

In [19]:
from cobra.solvers import gurobi_solver
from gurobipy import *
import numpy as np

F_TOL_FINAL = 0.04
F_TOL_START = 0.2
N_FS =5
F_TOLS = np.linspace(F_TOL_START, F_TOL_FINAL, N_FS)
F_TOLS

array([0.2 , 0.16, 0.12, 0.08, 0.04])

### TOOD: bisection on F_TOL instead of linear

In [20]:
%%time

import time

toc = 0.
sol_dict = {}
for ind, F_TOL in enumerate(F_TOLS):
    # ind = 0
    # F_TOL = F_TOL_FINAL
    for mdl_ind,mdl in iteritems(stacker.model_dict):
        mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
        rxn_mu = mdl.reactions.get_by_id(mu_id+'_%s'%mdl_ind)    
        rxn_mu.lower_bound = mu_meas*(1-F_TOL)
        rxn_mu.upper_bound = mu_meas*(1+F_TOL)
        mu_crowd0 = np.nan
        print('Condition=%s. Initial mu_crowd=%g. Fitting within %g%% of measured: %g <= mu <= %g' % (mdl_ind, mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, rxn_mu.upper_bound))

    milp = gurobi_solver.create_problem(stacker.model)
    milp.ModelSense = GRB.MINIMIZE
    milp.Params.IntFeasTol = 1e-9
    milp.Params.OutputFlag = 1
    # milp.Params.FeasibilityTol = 1e-9
    # milp.Params.OptimalityTol = 1e-9
    milp.Params.NodefileStart = 3   # Start writing nodes to disk if x GB RAM exceeded (for each thread)
    milp.Params.TimeLimit = 2*3600  # Time limit in seconds    
    #milp.Params.Method = 2 # Barrier
    #milp.Params.NodeMethod = 2 # Barrier slower for NodeMethod
    #milp.Params.BarHomogeneous = 1
    #***************************
    milp.Params.NumericFocus = 1
    milp.Params.ScaleFlag = 0
    milp.Params.Presolve  = 0
    #***************************
    tic = time.time()
    milp.optimize()
    toc = time.time()-tic

    # If feasible solution found, use it as MILP start
    if milp.SolCount > 0:
        print('#'*40)
        print('Setting MIP warm-start values.')
        for v in milp.getVars():
            v.Start = v.X
        sol_dict[ind] = {'x_dict':{v.VarName:v.X for v in milp.getVars()}, 'F_TOL':F_TOL, 'time':toc}

Condition=glucose. Initial mu_crowd=nan. Fitting within 20% of measured: 0.592 <= mu <= 0.888
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter NodefileStart to 3.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 7200.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter NumericFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter ScaleFlag to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Optimize a model with 135291 rows, 54848 columns and 345859 nonzeros
Variable types: 40722 continuous, 14126 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-12, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 1e+03]
  RHS range        [5e-03, 1e+03]
Variable types: 40603 continuous, 14245 integer (14126 binary)

Root simplex l

In [21]:
[(k, s['F_TOL'], s['time']) for k,s in iteritems(sol_dict)]

[(0, 0.2, 475.24176692962646),
 (1, 0.16, 29.735055923461914),
 (2, 0.12000000000000001, 38.473843812942505),
 (3, 0.08000000000000002, 60.570982933044434)]

In [22]:
best_ind = len(sol_dict)-1
best_ind

3

In [23]:
x_dict = {r.id:sol_dict[best_ind]['x_dict'][str(i)] for i,r in enumerate(stacker.model.reactions)}
F_TOL = sol_dict[best_ind]['F_TOL']

In [24]:
yopt = [x_dict[rxn.id] for rxn in stacker.model.reactions.query('binary_')]
print('Number of non-zero binaries: %g' % sum(yopt))
for mdl_ind,mdl in iteritems(stacker.model_dict):
    dfi = df_meas[ df_meas.substrate==mdl_ind]
    mu_measi = dfi.growth_rate_1_h.iloc[0]
    rxn_mu = stacker.model.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    muopt = x_dict[rxn_mu.id]
    muL = mu_measi*(1-F_TOL)
    muU = mu_measi*(1+F_TOL)
    print('%s: Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mdl_ind, mu_crowd0, 100*F_TOL, muL, muopt, muU))

Number of non-zero binaries: 2018
glucose: Initial mu_crowd=nan. Fitted within 8%: 0.6808 <= 0.776975 <= 0.7992


In [36]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = x_dict[yid]            
            if abs(y)>1e-10:
                #print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))
                pass
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

In [43]:
kfit_changed = []
for k,v in iteritems(kfit_dict):
    rxn0 = mdl_ref.reactions.get_by_id(k)
    a0   = rxn0.metabolites[mdl_ref.metabolites.crowding]    
    perr = abs(v-a0)/a0
    if perr>1e-6:
        kfit_changed.append((k,v,perr))
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
kfit_changed

Changed keffs: 2/2018


[(u'CYTBO3_4pp', 0.00020733815370597105, 7.0),
 (u'PGK_abs', 0.001207922009789367, 7.0)]

### Plug back in to be sure

In [44]:
#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:
    ijofit  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')
    crowding = ijofit.metabolites.crowding
    
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit})
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.get_by_id(mu_id).x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.get_by_id(mu_id).x
    err0= 100*(mu_unfiti-mu_measi)/mu_measi
    err = 100*(mu_fiti - mu_measi)/mu_measi
    derr= 100*(abs(err)-abs(err0))/abs(err0)
    print('Cond=%s. mu_meas=%g. mu_sim=%g (Error=%.3g%%) (unfit=%g, error=%.3g%%). (%.3g%% change)' % (csrc, mu_measi, mu_fiti, err, mu_unfiti, err0, derr))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))

Cond=glucose. mu_meas=0.74. mu_sim=0.754508 (Error=1.96%) (unfit=0.982372, error=32.8%). (-94% change)
	EX_glc__D_e uptake=-10
	EX_malt_e uptake=0
	EX_man_e uptake=0
	EX_fru_e uptake=0
	EX_gal_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_glyc_e uptake=0
	EX_lac__L_e uptake=0
	EX_ac_e uptake=0


Cond=glucose. mu_meas=0.74. mu_sim=0.619085 (Error=-16.3%) (unfit=0.982372, error=32.8%). (-50.1% change)